In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

Data preparation

In [96]:
data = pd.read_excel('data/Macro_data.xlsx', index_col=0).reset_index()
data = data.rename(columns={"TIME": "date"})
data['date'] = pd.to_datetime(data['date'])
data = data[(data['date'] >= "1970-01-01") & (data['date'] <= "2001-01-01")]

In [97]:
data.dropna(inplace=True)
data.head()

,date,GDP Nominal,GDP Real,Private Consumtion,Investments,Residential,Non-Residential,Changes in inventories,Cumulated Chnge in iventories,Total export,Total import,Government Consumption Expenditures,GDP Deflator,Consumption Deflator,"CPI, Index",Industrial production Index,Share prices,Unemployment,Personal Consumprion Expenditure
0,1970-01-01,1.051200e+12,4.939759e+12,1.579212e+11,5.759530e+11,4.249300e+10,1.237960e+11,1.823000e+09,1823000000,5.703100e+10,5.351700e+10,2.478890e+11,0.212804,17.416415,38.100000,37.9469,6.34526,0.044,1.895000e+12
1,1970-04-01,1.067375e+12,4.946770e+12,1.603925e+11,5.772050e+11,4.136500e+10,1.250080e+11,5.081000e+09,6904000000,6.041100e+10,5.520400e+10,2.491430e+11,0.215772,17.629837,38.633333,37.6830,5.56439,0.049,1.924700e+12
2,1970-07-01,1.086059e+12,4.992357e+12,1.633705e+11,5.865980e+11,4.256600e+10,1.262550e+11,5.083000e+09,11987000000,6.051900e+10,5.643100e+10,2.545840e+11,0.217544,17.806946,39.033333,37.4482,5.48283,0.054,1.960500e+12
3,1970-10-01,1.088608e+12,4.938857e+12,1.650402e+11,5.554540e+11,4.721800e+10,1.235200e+11,-3.984000e+09,8003000000,6.087300e+10,5.788800e+10,2.587080e+11,0.220417,18.027467,39.600000,37.3145,6.01062,0.061,1.980500e+12
4,1971-01-01,1.135156e+12,5.072996e+12,1.697965e+11,6.202120e+11,5.096100e+10,1.262810e+11,1.225400e+10,20257000000,6.322100e+10,5.865100e+10,2.619050e+11,0.223764,18.182825,39.933333,37.4892,6.80859,0.060,2.037500e+12


Auxiliary func

In [98]:
def dickey_fuller_test(series):
    res = adfuller(series)
    print("Adfuller for  {} | adf: {:.3f} | p-value: {:.3f}".format(series.name, res[0], res[1]))
    if res[1] > 0.05:
        print('Non-stationary')
    else:
        print('Stationary')


def bkfilter(series, params):
    return sm.tsa.filters.bkfilter(series, params[0], params[1], params[2])


def hodrick_prescott_filter(series):
    return sm.tsa.filters.hpfilter(series)


def crosscorr(x, y, lag=0):
    return x.corr(y.shift(lag))

Wrapping cross correlations into func

In [99]:
def calculate_cross_corrs(series, col):
    series[f"{col}_ln"] = np.log(series[col])
    for params in [(6, 40, 12), (6, 40, 8), (6, 32, 8)]:
        series[f"{col}_ln_BXKG {params}"] = bkfilter(series[f"{col}_ln"], params)
    cycle, trend = hodrick_prescott_filter(series[f"{col}_ln"])
    series[f'{col}_ln_HP'] = cycle
    column_to_show = [f"{col}_ln_HP", f"{col}_ln_BXKG (6, 40, 8)",  f"{col}_ln_BXKG (6, 32, 8)", f"{col}_ln_BXKG (6, 40, 12)"]

    series.dropna(inplace=True)
    for column in series.columns.values:
        dickey_fuller_test(series[column])

    stds = series[column_to_show].std().round(3).reset_index()
    corrs = series[column_to_show].corr().round(3).reset_index()
    res = stds.merge(corrs)
    return res.rename(columns={0: "std"})

Adfuller + cross correlations

In [100]:
gdp_real_matrix = calculate_cross_corrs(data, 'GDP Real')
gdp_real_matrix

Adfuller for  date | adf: -0.183 | p-value: 0.940
Non-stationary
Adfuller for  GDP Nominal | adf: 4.001 | p-value: 1.000
Non-stationary
Adfuller for  GDP Real | adf: 2.007 | p-value: 0.999
Non-stationary
Adfuller for  Private Consumtion | adf: 3.899 | p-value: 1.000
Non-stationary
Adfuller for  Investments | adf: 0.985 | p-value: 0.994
Non-stationary
Adfuller for  Residential | adf: 0.045 | p-value: 0.962
Non-stationary
Adfuller for  Non-Residential | adf: 1.121 | p-value: 0.995
Non-stationary
Adfuller for  Changes in inventories | adf: -4.106 | p-value: 0.001
Stationary
Adfuller for  Cumulated Chnge in iventories | adf: 2.575 | p-value: 0.999
Non-stationary
Adfuller for  Total export | adf: 1.876 | p-value: 0.998
Non-stationary
Adfuller for  Total import | adf: 3.514 | p-value: 1.000
Non-stationary
Adfuller for  Government Consumption Expenditures | adf: 0.978 | p-value: 0.994
Non-stationary
Adfuller for  GDP Deflator | adf: -1.894 | p-value: 0.335
Non-stationary
Adfuller for  Consump

,index,std,GDP Real_ln_HP,"GDP Real_ln_BXKG (6, 40, 8)","GDP Real_ln_BXKG (6, 32, 8)","GDP Real_ln_BXKG (6, 40, 12)"
0,GDP Real_ln_HP,0.017,1.000,0.924,0.918,0.968
1,"GDP Real_ln_BXKG (6, 40, 8)",0.013,0.924,1.000,1.000,0.940
2,"GDP Real_ln_BXKG (6, 32, 8)",0.013,0.918,1.000,1.000,0.934
3,"GDP Real_ln_BXKG (6, 40, 12)",0.018,0.968,0.940,0.934,1.000


In [101]:
consumption_matrix = calculate_cross_corrs(data, 'Personal Consumprion Expenditure ')
consumption_matrix

Adfuller for  date | adf: -0.157 | p-value: 0.943
Non-stationary
Adfuller for  GDP Nominal | adf: 3.076 | p-value: 1.000
Non-stationary
Adfuller for  GDP Real | adf: 0.985 | p-value: 0.994
Non-stationary
Adfuller for  Private Consumtion | adf: 4.533 | p-value: 1.000
Non-stationary
Adfuller for  Investments | adf: -0.558 | p-value: 0.880
Non-stationary
Adfuller for  Residential | adf: -0.814 | p-value: 0.815
Non-stationary
Adfuller for  Non-Residential | adf: -0.501 | p-value: 0.892
Non-stationary
Adfuller for  Changes in inventories | adf: -3.954 | p-value: 0.002
Stationary
Adfuller for  Cumulated Chnge in iventories | adf: 0.797 | p-value: 0.992
Non-stationary
Adfuller for  Total export | adf: 1.371 | p-value: 0.997
Non-stationary
Adfuller for  Total import | adf: 1.728 | p-value: 0.998
Non-stationary
Adfuller for  Government Consumption Expenditures | adf: 0.186 | p-value: 0.971
Non-stationary
Adfuller for  GDP Deflator | adf: -2.368 | p-value: 0.151
Non-stationary
Adfuller for  Cons

,index,std,Personal Consumprion Expenditure _ln_HP,"Personal Consumprion Expenditure _ln_BXKG (6, 40, 8)","Personal Consumprion Expenditure _ln_BXKG (6, 32, 8)","Personal Consumprion Expenditure _ln_BXKG (6, 40, 12)"
0,Personal Consumprion Expenditure _ln_HP,0.008,1.000,0.891,0.888,0.890
1,"Personal Consumprion Expenditure _ln_BXKG (6, ...",0.007,0.891,1.000,1.000,0.956
2,"Personal Consumprion Expenditure _ln_BXKG (6, ...",0.006,0.888,1.000,1.000,0.950
3,"Personal Consumprion Expenditure _ln_BXKG (6, ...",0.009,0.890,0.956,0.950,1.000


In [102]:
cpi_matrix = calculate_cross_corrs(data, 'CPI, Index')
cpi_matrix

Adfuller for  date | adf: -0.277 | p-value: 0.929
Non-stationary
Adfuller for  GDP Nominal | adf: 0.236 | p-value: 0.974
Non-stationary
Adfuller for  GDP Real | adf: -0.081 | p-value: 0.951
Non-stationary
Adfuller for  Private Consumtion | adf: 0.726 | p-value: 0.990
Non-stationary
Adfuller for  Investments | adf: -1.087 | p-value: 0.720
Non-stationary
Adfuller for  Residential | adf: -1.163 | p-value: 0.689
Non-stationary
Adfuller for  Non-Residential | adf: -1.263 | p-value: 0.646
Non-stationary
Adfuller for  Changes in inventories | adf: -4.105 | p-value: 0.001
Stationary
Adfuller for  Cumulated Chnge in iventories | adf: 0.324 | p-value: 0.978
Non-stationary
Adfuller for  Total export | adf: 0.844 | p-value: 0.992
Non-stationary
Adfuller for  Total import | adf: -0.468 | p-value: 0.898
Non-stationary
Adfuller for  Government Consumption Expenditures | adf: -0.083 | p-value: 0.951
Non-stationary
Adfuller for  GDP Deflator | adf: -1.049 | p-value: 0.735
Non-stationary
Adfuller for  C

,index,std,"CPI, Index_ln_HP","CPI, Index_ln_BXKG (6, 40, 8)","CPI, Index_ln_BXKG (6, 32, 8)","CPI, Index_ln_BXKG (6, 40, 12)"
0,"CPI, Index_ln_HP",0.016,1.000,0.951,0.947,0.976
1,"CPI, Index_ln_BXKG (6, 40, 8)",0.010,0.951,1.000,1.000,0.966
2,"CPI, Index_ln_BXKG (6, 32, 8)",0.009,0.947,1.000,1.000,0.963
3,"CPI, Index_ln_BXKG (6, 40, 12)",0.017,0.976,0.966,0.963,1.000
